In [1]:
import torch
from BatchGenerator import get_batch

In [ ]:
#Reading the contents of the file
with open('HighSchool Physics.txt', encoding='utf-8') as file:
    text = file.read()

In [ ]:
#Creating an array of all the characters in the file
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [ ]:
#Mapping the characters to integers and vice versa
stoi = { ch:i  for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
#Creating a data tensor
data = torch.tensor(encode(text), dtype=torch.long)

#Splitting the data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
torch.manual_seed(1337)
batch_size = 4 #Number of independent sequences processed in parallel
block_size = 8 #Maximum context length for predictions

xb, yb = get_batch('train', train_data, val_data, batch_size, block_size)